In [1]:

import acquire
import prep

# for presentation purposes
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# visualize 
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

### Acquire

In [2]:
# acquire data
data = acquire.epi_category('711238')

In [ ]:
series_id_list, series_name_list = acquire.data_manipulation(data)

In [ ]:
df_list = acquire.build_df_list_rename(series_id_list, series_name_list)

### prep

In [ ]:
df = prep.prep_data(df_list)

In [ ]:
df

In [ ]:
df = prep.feat_eng(df)

#### explore

In [ ]:
df = prep.fossil_fuels(df)

In [ ]:
df.head()

In [ ]:
# plotting defaults
plt.rc('figure', figsize=(11, 5))
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=16)

print('Date Range:', df.index.min(), 'to', df.index.max())
df.head()

df.shape[0]

In [ ]:
df.total_fossil_fuels_production_monthly.describe()

seasonality: a repeated cycle in the data. Occurs at a fixed frequency. In our weather data there is yearly and daily seasonality
trend: long term upwards or downwards movement
cycle: Some arbitrary chunk of time, usually longer than a season, or consists of multiple seasons

Train-Test Split

Ideally all splits contain a season
Human-based: use the last year as test
Percentage based: use the last 20% as test
Cross Validate: break data up into slices and use successive slices as train and test repeatedly (sklearn.model_selection.TimeSeriesSplit)

In [ ]:
# # Percentage-Based
# train_size = .70
# n = df.shape[0]
# test_start_index = round(train_size * n)

# train = df[:test_start_index] # everything up (not including) to the test_start_index
# test = df[test_start_index:] # everything from the test_start_index to the end

# plt.plot(train.index, train.total_fossil_fuels_production_monthly)
# plt.plot(test.index, test.total_fossil_fuels_production_monthly)

In [ ]:
# Human-Based
train = df.loc['2005':'2014'] # includes 2010
test = df.loc['2015':'2021']

plt.plot(train.index, train.total_fossil_fuels_production_monthly)
plt.plot(test.index, test.total_fossil_fuels_production_monthly)

Visualization

In [ ]:
y = train.total_fossil_fuels_production_monthly
x = train.total_fossil_fuels_consumption_monthly
z = train.fossil_fuels_difference
y.head()

In [ ]:
y.plot.hist()

In [ ]:
x.plot.hist()

In [ ]:
z.plot.hist()

In [ ]:
ax = train.groupby(train.index.month).total_fossil_fuels_production_monthly.mean().plot.bar()
ax.tick_params('x', rotation=0)

In [ ]:
ax = train.groupby(train.index.month).total_fossil_fuels_consumption_monthly.mean().plot.bar()
ax.tick_params('x', rotation=0)

In [ ]:
ax = train.groupby(train.index.month).fossil_fuels_difference.mean().plot.bar()
ax.tick_params('x', rotation=0)

In [ ]:
train['month'] = train.index.month_name()

In [ ]:
sns.boxplot(data=train, y='total_fossil_fuels_production_monthly', x='month')
# rotate x-axis labels
plt.xticks(rotation=45)

In [ ]:
sns.boxplot(data=train, y='total_fossil_fuels_consumption_monthly', x='month')
# rotate x-axis labels
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.boxplot(data=train, y='fossil_fuels_difference', x='month')
# rotate x-axis labels
plt.xticks(rotation=45)

In [ ]:
y.plot(alpha=.2, label='Monthly')
y.resample('3M').mean().plot(alpha=.5, label='Tri-Monthly')
y.resample('6M').mean().plot(alpha=.8, label='Bi-Yearly')
y.resample('Y').mean().plot(label='Yearly')
y.resample('5Y').mean().plot(label='Bi-Decade')
plt.legend()
# add labels
plt.xlabel('Date')
plt.ylabel('Production (MtCO2)')
plt.title('Fossil Fuels Production')

In [ ]:
x.plot(alpha=.2, label='Monthly')
x.resample('3M').mean().plot(alpha=.5, label='Tri-Monthly')
x.resample('6M').mean().plot(alpha=.8, label='Bi-Yearly')
x.resample('Y').mean().plot(label='Yearly')
x.resample('5Y').mean().plot(label='Bi-Decade')
plt.legend()
# add labels
plt.xlabel('Date')
plt.ylabel('Consumption (MtCO2)')
plt.title('Fossil Fuels Consumption')

In [ ]:
z.plot(alpha=.2, label='Monthly')
z.resample('3M').mean().plot(alpha=.5, label='Tri-Monthly')
z.resample('6M').mean().plot(alpha=.8, label='Bi-Yearly')
z.resample('Y').mean().plot(label='Yearly')
z.resample('5Y').mean().plot(label='Bi-Decade')
plt.legend()
# add labels
plt.xlabel('Date')
plt.ylabel('Difference (MtCO2)')
plt.title('Fossil Fuels Difference in Production and Consumption')

Change in difference over time

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(16, 9))

ax = y.resample('M').mean().diff(12).plot(ax=ax1)
ax1.hlines(0, *ax1.get_xlim(), color='black', ls=':')
ax1.set(title='Difference from the same month last year')

y.resample('M').mean().plot(ax=ax2)
ax2.set(title='Fossil Fuel Production over time')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(16, 9))

ax = x.resample('M').mean().diff(12).plot(ax=ax1)
ax1.hlines(0, *ax1.get_xlim(), color='black', ls=':')
ax1.set(title='Difference from the same month last year')

x.resample('M').mean().plot(ax=ax2)
ax2.set(title='Fossil Fuel Consumption over time')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(16, 9))

ax = z.resample('M').mean().diff(12).plot(ax=ax1)
ax1.hlines(0, *ax1.get_xlim(), color='black', ls=':')
ax1.set(title='Difference from the same month last year')

z.resample('M').mean().plot(ax=ax2)
ax2.set(title='Difference in consuption/production over time')

Seasonal Plot

xaxis: month
yaxis: y
color: year

In [ ]:
y.groupby([y.index.year, y.index.month]).mean().unstack(0).plot(title='Seasonal Plot')


In [ ]:
x.groupby([x.index.year, x.index.month]).mean().unstack(0).plot(title='Seasonal Plot')


In [ ]:
z.groupby([z.index.year, z.index.month]).mean().unstack(0).plot(title='Seasonal Plot')


Seasonal Subseries Plot

In [ ]:
table = y.groupby([y.index.year, y.index.month]).mean().unstack()
table

In [ ]:
for month_number, subset in table.iteritems():
    print(month_number)

In [ ]:
for number, letter in zip([1, 2, 3], ['a', 'b', 'c']):
    print(number, letter)

In [ ]:
ax.get_xlim()

In [ ]:
table = y.groupby([y.index.year, y.index.month]).mean().unstack()

fig, axs = plt.subplots(1, 12, sharey=True, sharex=True, figsize=(16, 5))

for ax, (month, subset) in zip(axs, table.iteritems()):
    subset.plot(ax=ax, title=month)
    ax.hlines(subset.mean(), *ax.get_xlim(), color='black', ls='--')
    ax.set(xlabel='')
    ax.tick_params('x', rotation=30)
    
fig.suptitle('Seasonal Subseries Plot') # super-title for the overall figure
# fig.tight_layout()
fig.subplots_adjust(wspace=0)

In [ ]:
table = x.groupby([x.index.year, x.index.month]).mean().unstack()

fig, axs = plt.subplots(1, 12, sharey=True, sharex=True, figsize=(16, 5))

for ax, (month, subset) in zip(axs, table.iteritems()):
    subset.plot(ax=ax, title=month)
    ax.hlines(subset.mean(), *ax.get_xlim(), color='black', ls='--')
    ax.set(xlabel='')
    ax.tick_params('x', rotation=30)
    
fig.suptitle('Seasonal Subseries Plot') # super-title for the overall figure
# fig.tight_layout()
fig.subplots_adjust(wspace=0)

In [ ]:
table = z.groupby([z.index.year, z.index.month]).mean().unstack()

fig, axs = plt.subplots(1, 12, sharey=True, sharex=True, figsize=(16, 5))

for ax, (month, subset) in zip(axs, table.iteritems()):
    subset.plot(ax=ax, title=month)
    ax.hlines(subset.mean(), *ax.get_xlim(), color='black', ls='--')
    ax.set(xlabel='')
    ax.tick_params('x', rotation=30)
    
fig.suptitle('Seasonal Subseries Plot') # super-title for the overall figure
# fig.tight_layout()
fig.subplots_adjust(wspace=0)

Lag Plot

In [ ]:
df.head()

In [ ]:
train['y(t + 1)'] = train.total_fossil_fuels_production_monthly	.shift(-1)
train.head()

In [ ]:
ax = train.plot.scatter(x='total_fossil_fuels_production_monthly', y='y(t + 1)')
ax.set(xlabel='t', ylabel='t + 1')

In [ ]:
monthly = train.resample('M').mean().drop(columns='y(t + 1)')
monthly['the_next_month'] = monthly.total_fossil_fuels_production_monthly.shift(-1)
monthly = monthly.rename(columns={'total_fossil_fuels_production_monthly': 'this_month'})
monthly.plot.scatter(x='this_month', y='the_next_month')
monthly

In [ ]:
monthly = train.resample('M').mean().drop(columns='y(t + 1)')
monthly['6_months_out'] = monthly.total_fossil_fuels_production_monthly.shift(-6)
monthly = monthly.rename(columns={'total_fossil_fuels_production_monthly': 'this_month'})
monthly.plot.scatter(x='this_month', y='6_months_out')

In [ ]:
monthly = train.resample('M').mean().drop(columns='y(t + 1)')
monthly['12_months_out'] = monthly.total_fossil_fuels_production_monthly.shift(-12)
monthly = monthly.rename(columns={'total_fossil_fuels_production_monthly': 'this_month'})
monthly.plot.scatter(x='this_month', y='12_months_out')

In [ ]:
pd.plotting.lag_plot(train.total_fossil_fuels_production_monthly.resample('M').mean(), lag=60)

Autocorrelation Plot

What is pearson's r as a function of the lag time?

autocorrelation: a series correlation with itself
can help to identify seasonality

In [ ]:
from scipy import stats

In [ ]:
monthly

In [ ]:
lag = 1
monthly.total_fossil_fuels_consumption_monthly.iloc[:-lag].shape, monthly.total_fossil_fuels_consumption_monthly.shift(-lag).dropna().shape

In [ ]:
monthly = train.resample('M').mean()
s = pd.Series({
    lag: stats.pearsonr(
        monthly.total_fossil_fuels_consumption_monthly.iloc[:-lag], monthly.total_fossil_fuels_consumption_monthly.shift(-lag).dropna()
    )[0]
    for lag in range(1, 12*3 + 1)
})

In [ ]:
pd.plotting.autocorrelation_plot(train.total_fossil_fuels_consumption_monthly.resample('M').mean())

Seasonal Decomposition

In [ ]:
y = train.total_fossil_fuels_production_monthly.resample('M').mean()

result_p = sm.tsa.seasonal_decompose(y)
decomposition = pd.DataFrame({
    'y': result_p.observed,
    'trend': result_p.trend,
    'seasonal': result_p.seasonal,
    'resid': result_p.resid,
})
decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot()

In [ ]:
decomposition.y.plot()

In [ ]:
x = train.total_fossil_fuels_consumption_monthly.resample('M').mean()

result_c = sm.tsa.seasonal_decompose(x)
decomposition = pd.DataFrame({
    'x': result_c.observed,
    'trend': result_c.trend,
    'seasonal': result_c.seasonal,
    'resid': result_c.resid,
})
decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot()

In [ ]:
decomposition.x.plot()

In [ ]:
z = train.fossil_fuels_difference.resample('M').mean()

result_d = sm.tsa.seasonal_decompose(z)
decomposition = pd.DataFrame({
    'z': result_d.observed,
    'trend': result_d.trend,
    'seasonal': result_d.seasonal,
    'resid': result_d.resid,
})
decomposition['trend_centered'] = decomposition.trend - decomposition.trend.mean()
decomposition[['trend_centered', 'seasonal', 'resid']].plot()

In [ ]:
decomposition.z.plot()

In [ ]:
result_p.plot()
None
result_c.plot()
None
result_d.plot()
None

In [ ]:
# Simulation Demo
import numpy as np

x = np.linspace(0, 8 * np.pi, 200)
seasonal = np.sin(x)
trend = np.linspace(0, 1, 200)
resid = np.random.rand(200)

y = seasonal + trend + resid

plt.plot(x, y)